In [2]:
# ==========================================================
#  Wine Quality – Logistic Regression Predictor (PC version)
#  Entrena, evalúa y guarda el modelo + scaler para usar en Arduino.
#  También permite probar predicciones manuales en el PC.
# ==========================================================

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib, os   # joblib para guardar/cargar modelos, os para manejo de archivos y carpetas

# --- Rangos de atributos originales (útiles para validación al pedir datos al usuario) ---
RANGOS = {
    "red": {   # Rango válido de cada variable para vino tinto
        "fixed acidity":        (4.6, 15.9),
        "volatile acidity":     (0.12, 1.58),
        "citric acid":          (0.00, 1.00),
        "residual sugar":       (0.9, 15.5),
        "chlorides":            (0.012, 0.611),
        "free sulfur dioxide":  (1, 72),
        "total sulfur dioxide": (6, 289),
        "density":              (0.990, 1.004),
        "pH":                   (2.74, 4.01),
        "sulphates":            (0.33, 2.00),
        "alcohol":              (8.4, 14.9)
    },
    "white": { # Rango válido de cada variable para vino blanco
        "fixed acidity":        (3.8, 14.2),
        "volatile acidity":     (0.08, 1.10),
        "citric acid":          (0.00, 1.66),
        "residual sugar":       (0.6, 65.8),
        "chlorides":            (0.009, 0.346),
        "free sulfur dioxide":  (2, 289),
        "total sulfur dioxide": (9, 440),
        "density":              (0.987, 1.039),
        "pH":                   (2.72, 3.82),
        "sulphates":            (0.22, 1.08),
        "alcohol":              (8.0, 14.2)
    }
}

# ----------------------------------------------------------
# 1. Selección del dataset (tinto o blanco) según el usuario
# ----------------------------------------------------------
print("=== Selección de dataset ===")
print("1. Vino tinto (red)")
print("2. Vino blanco (white)")
op = input("Elige 1 o 2: ").strip()  # Lee la opción desde consola

# Se elige archivo y tipo de vino
if op == "2":
    file_name = "winequality-white.csv"
    wine_type = "white"
    print("\nUsando dataset de VINO BLANCO...")
else:
    file_name = "winequality-red.csv"
    wine_type = "red"
    print("\nUsando dataset de VINO TINTO...")

# ----------------------------------------------------------
# 2. Carga del dataset CSV seleccionado
# ----------------------------------------------------------
df = pd.read_csv(file_name, sep=";")   # El archivo usa ; como separador
print("\nPrimeras filas:\n", df.head(), "\n")  # Vista rápida de los primeros registros

# ----------------------------------------------------------
# 3. Preparación de datos para el modelo
# ----------------------------------------------------------
y = df["quality"]          # Variable objetivo: calidad del vino (1–10)
X = df.drop("quality", axis=1)  # Variables predictoras: las 11 características químicas

# Divide datos en entrenamiento (80%) y prueba (20%) estratificando por la clase
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Estandariza los datos (media 0, varianza 1) para mejorar el rendimiento del modelo
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)  # Ajusta y transforma el conjunto de entrenamiento
X_test_s  = scaler.transform(X_test)       # Usa el mismo escalador para el conjunto de prueba

# ----------------------------------------------------------
# 4. Entrenamiento del modelo de Regresión Logística
# ----------------------------------------------------------
model = LogisticRegression(
    multi_class="multinomial",  # Predicción multiclase
    max_iter=1000,              # Aumenta iteraciones para asegurar convergencia
    solver="lbfgs"              # Algoritmo de optimización recomendado para multiclase
)
model.fit(X_train_s, y_train)    # Entrena el modelo con datos normalizados

# ----------------------------------------------------------
# 5. Evaluación en el conjunto de prueba
# ----------------------------------------------------------
preds = model.predict(X_test_s)                     # Predicciones sobre datos de prueba
print("\nExactitud en test:", accuracy_score(y_test, preds))  # Mide precisión global

# ----------------------------------------------------------
# 6. Guardar modelo y escalador para uso posterior o en Arduino
# ----------------------------------------------------------
os.makedirs("modelo_wine", exist_ok=True)           # Crea carpeta si no existe
joblib.dump(model, "modelo_wine/wine_model.pkl")    # Guarda el modelo entrenado
joblib.dump(scaler, "modelo_wine/wine_scaler.pkl")  # Guarda el escalador
print("\nModelo y scaler guardados en carpeta 'modelo_wine'.")

# ----------------------------------------------------------
# 7. Exportar coeficientes a un archivo de texto para Arduino
#    (facilita copiar pesos, interceptos y clases)
# ----------------------------------------------------------
coefs = model.coef_           # Matriz de pesos (una fila por clase)
intercepts = model.intercept_ # Interceptos (sesgos) por clase
classes = model.classes_      # Etiquetas de las clases de calidad
with open("modelo_wine/modelo_arduino.txt", "w") as f:
    f.write("# Coeficientes para Logistic Regression\n")
    f.write("# Orden de features: " + ", ".join(X.columns) + "\n\n")
    f.write("CLASES: " + " ".join(map(str, classes)) + "\n\n")
    f.write("INTERCEPTOS:\n")
    f.write(" ".join(map(str, intercepts)) + "\n\n")
    f.write("COEFICIENTES (filas=clases, cols=features):\n")
    for row in coefs:
        f.write(" ".join(map(str, row)) + "\n")
print("Archivo 'modelo_arduino.txt' generado con pesos listos para Arduino.")

# ----------------------------------------------------------
# 8. Predicción manual interactiva en el PC
#    Permite al usuario ingresar 11 atributos y ver
#    la calidad predicha y la probabilidad de cada clase.
# ----------------------------------------------------------
features = list(X.columns)
print("\n=== Predicción manual ===")
print("Introduce los 11 atributos (ENTER vacío para salir).\n")

while True:
    user_vals = []
    for f in features:
        low, high = RANGOS[wine_type][f]          # Rango recomendado de cada atributo
        val = input(f"{f} (rango sugerido {low}–{high}): ")
        if val.strip() == "":                     # Salir si se presiona ENTER sin valor
            print("Saliendo.")
            exit()
        try:
            num = float(val)                      # Convierte a número flotante
        except ValueError:
            print("Número inválido, vuelve a intentar.")
            break
        if not (low <= num <= high):              # Advertencia si está fuera del rango típico
            print("⚠️ Valor fuera de rango (se aceptará de todas formas).")
        user_vals.append(num)
    else:  # Se ejecuta solo si no se usó 'break' en el bucle
        x_new = scaler.transform([user_vals])     # Aplica misma normalización que el entrenamiento
        pred_class = model.predict(x_new)[0]      # Predice la clase de calidad (1–10)
        probs = model.predict_proba(x_new)[0]     # Probabilidades para cada clase
        print(f"\nPredicción de calidad (1–10): {int(pred_class)}")
        print("Probabilidades por clase:")
        for c, p in zip(classes, probs):
            print(f"  Calidad {c}: {p*100:.2f}%")
        print()


=== Selección de dataset ===
1. Vino tinto (red)
2. Vino blanco (white)
Elige 1 o 2: 2

Usando dataset de VINO BLANCO...

Primeras filas:
    fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.0              0.27         0.36            20.7      0.045   
1            6.3              0.30         0.34             1.6      0.049   
2            8.1              0.28         0.40             6.9      0.050   
3            7.2              0.23         0.32             8.5      0.058   
4            7.2              0.23         0.32             8.5      0.058   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 45.0                 170.0   1.0010  3.00       0.45   
1                 14.0                 132.0   0.9940  3.30       0.49   
2                 30.0                  97.0   0.9951  3.26       0.44   
3                 47.0                 186.0   0.9956  3.19       0.40   
4                 47.0

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



Exactitud en test: 0.5489795918367347

Modelo y scaler guardados en carpeta 'modelo_wine'.
Archivo 'modelo_arduino.txt' generado con pesos listos para Arduino.

=== Predicción manual ===
Introduce los 11 atributos (ENTER vacío para salir).

fixed acidity (rango sugerido 3.8–14.2): 
Saliendo.
Número inválido, vuelve a intentar.


KeyboardInterrupt: Interrupted by user

In [ ]:
print("mean_:", scaler.mean_)
print("scale_:", scaler.scale_)

mean_: [ 8.30648944  0.52807271  0.27387803  2.54358874  0.08893432 16.12783425
 47.28068804  0.99673525  3.30858483  0.66367475 10.42444618]
scale_: [1.69466258e+00 1.77640746e-01 1.94825888e-01 1.37515865e+00
 5.09613176e-02 1.04694667e+01 3.33217514e+01 1.85141936e-03
 1.51539761e-01 1.78290748e-01 1.06525516e+00]
